In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import copy

# Settings for printing numpy arrays
np.set_printoptions(precision=4, edgeitems=50, suppress=True)
np.core.arrayprint._line_width = 250

# Sensitivity in tableau format

Below are defined some cases when we modify an specific value from our initial Linear Programming Problem. In order to do that, we are going to retake the previous work of the Simplex Method and the Dual Simplex Method in tableau format.

In [3]:
def simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, itera = 1):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
        
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    ZXN = np.ravel(tableau[0, [i + 1 for i in non_basic_indexes]])
    XBXN = tableau[1:, [i + 1 for i in non_basic_indexes]]
    ZRHS = np.array([tableau[0, n_plus_m + 1]])
    XBRHS = (tableau[1:, n_plus_m + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n_plus_m - m)))
    
    print("\nEntry tableau")
    print(tableau)
    
    z_minus_c_max = max(ZXN)
    k = np.argmax(ZXN)
    
    # Validating if we stop or not the optimization
    if z_minus_c_max <= 0:   # We stop
        
        # Printing the final results
        print("\n\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal BFS is")
        results_ls = [str(round(i, 4)) for i in b_]
        print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        print("\nWith performance z =", perf_z, "\n\n")
        
        indexes_variables = [int(var[1:]) for var in basic_variables + non_basic_variables]
        dict_solution = dict(zip(indexes_variables, [b_i for b_i in b_]))
        sorted_dict_solution = dict(sorted(dict_solution.items(), key = lambda x:x[0]))
        solution = np.array(list(sorted_dict_solution.values()))
        #print(solution)
        return solution
        
    else:   # We continue
        
        # Calculating and printing the y_ki's
        y_k = np.ravel(XBXN[:, k])
        
        print("\nk =", k + 1, "-> column no.", k + 1, "of x_N part (", non_basic_variables[k], ")")
        print("y_k")
        print(y_k)
        
        # Analyzing if the optimal BFS is or not boundable by the condition y_k > 0
        flag = True
        aux_counter = 0
        for y_i in y_k:
            if y_i > 0:
                aux_counter = aux_counter + 1      
        if aux_counter == 0:
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_r_ls = list()
            for i in range(m):
                if y_k[i] > 0:
                    quot_ls.append(b_[i] / y_k[i])
                    index_r_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_r_arr = np.array(index_r_ls)
            r = index_r_arr[np.argmin(quot_arr)]
            x_Br = min(quot_arr)
            print("\nr =", r + 1, "-> column no.", r + 1, "of x_B part (", basic_variables[r], ")")
            print("x_Br")
            print(x_Br)
            
            # Pivoting
            pivot = y_k[r]
            print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, non_basic_indexes[k] + 1] * tableau[r + 1, :]
            
            print("\nPivoted tableau")
            print(tableau)
            # Exchanging the indexes
            print("\n\033[94m", non_basic_variables[k], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_indexes[r]
            basic_indexes[r] = non_basic_indexes[k]
            non_basic_indexes[k] = aux_vars
            # Exchanging columns in tableau
            #aux_column = tableau[:, m + k + 1]
            #tableau[:, m + k + 1] =  tableau[:, 1 + r]
            #tableau[:, 1 + r] = aux_column
            
            # Recursive call
            return simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, itera + 1)

        else:   # Not boundable
            
            print("\n\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe optimal BFS is not boundable\n\n")
            
            return False
            

In [4]:
def dual_simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, slack_indexes, itera = 1):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
    
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    ZX = np.ravel(tableau[0, 1:n_plus_m + 1])
    XBX = tableau[1:, 1:n_plus_m + 1]
    ZRHS = np.array([tableau[0, n_plus_m + 1]])
    XBRHS = (tableau[1:, n_plus_m + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n_plus_m - m)))
    
    print("\nEntry tableau")
    print(tableau)
    
    positive_b = True
    for b_i in np.ravel(XBRHS):
        if b_i < 0:
            positive_b = False
            break
    
    # Validating if we stop or not the optimization
    if positive_b:   # We stop
        
        # Printing the final results
        print("\n\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal primal solution is")
        results_primal_ls = [str(round(i, 4)) for i in b_]
        print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_primal_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        print("\nWith performance z =", perf_z)
        print("\nAnd the optimal dual solution is")
        results_dual_ls = np.ravel(-1 * tableau[0, [i + 1 for i in slack_indexes]]).tolist()
        results_dual_ls_f = [str(round(i, 4)) for i in results_dual_ls]
        print("\033[92m[ " + ', '.join(['w' + str(i + 1) for i in range(m)]) + " ] = [ " + ', '.join(results_dual_ls_f) + " ]\033[0m\n\n")
        
        indexes_variables = [int(var[1:]) for var in basic_variables + non_basic_variables]
        dict_primal_solution = dict(zip(indexes_variables, [b_i for b_i in b_]))
        sorted_dict_primal_solution = dict(sorted(dict_primal_solution.items(), key = lambda x:x[0]))
        primal_solution = np.array(list(sorted_dict_primal_solution.values()))
        
        dict_dual_solution = dict(zip([i for i in range(m)], results_dual_ls))
        sorted_dict_dual_solution = dict(sorted(dict_dual_solution.items(), key = lambda x:x[0]))
        dual_solution = np.array(list(sorted_dict_dual_solution.values()))
        
        #print(solution)
        return primal_solution, dual_solution
        
    else:   # We continue
        
        br = min(XBRHS)
        r = np.argmin(XBRHS)
        
        # Calculating and printing the y_ri's
        y_r = np.ravel(XBX[r, :])
        
        print("\nr =", r + 1, "-> row no.", r + 1, "of x_B part (", basic_variables[r], ")")
        print("y_r")
        print(y_r)
        
        # Analyzing if the Dual Problem is or not boundable by the condition y_r >= 0
        flag = True
        aux_counter = 0
        for y_i in y_r:
            if y_i >= 0:
                aux_counter += 1      
        if aux_counter == len(y_r):
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_k_ls = list()
            for i in range(n_plus_m):
                if y_r[i] < 0:
                    quot_ls.append(ZX[i] / y_r[i])
                    index_k_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_k_arr = np.array(index_k_ls)
            k = index_k_arr[np.argmin(quot_arr)]
            min_quot = min(quot_arr)
            print("\nk =", k + 1, "-> column no.", k + 1, "of x part (", variables[k], ")")
            print("min_quotient")
            print(min_quot)
            
            # Pivoting
            pivot = y_r[k]
            print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, k + 1] * tableau[r + 1, :]
            
            print("\nPivoted tableau")
            print(tableau)
            # Exchanging the indexes
            k_relative = 0
            for i in range(len(non_basic_indexes)):
                if k == non_basic_indexes[i]:
                    k_relative = i
                    break
            print("\n\033[94m", non_basic_variables[k_relative], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_indexes[r]
            basic_indexes[r] = non_basic_indexes[k_relative]
            non_basic_indexes[k_relative] = aux_vars
            
            # Recursive call
            return dual_simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, slack_indexes, itera + 1)

        else:   # Not boundable
            
            print("\n\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe Dual Problem is not boundable, thus the primal feasible region is empty\n\n")
            
            return False, False
            

In [5]:
def construct_tableau(A, b, c, basic_indexes):
    m, n_plus_m = np.shape(A)
    B = A[:m, basic_indexes]
    c_b = c[basic_indexes]
    b_ = np.linalg.inv(B) @ b

    one_array = np.ones(1, dtype=int)
    zero_array_v = np.ravel(np.zeros(m, dtype=int)).reshape(1, m)
    ZX = np.ravel((c_b @ np.linalg.inv(B) @ A) - c)
    XBX = np.linalg.inv(B) @ A
    ZRHS = np.full((1), c_b @ np.ravel(b_))
    XBRHS = b_.reshape(1, m)

    tableau = np.vstack((np.concatenate((one_array, ZX, ZRHS), axis = 0).reshape(1, n_plus_m + 2),
                  np.concatenate((zero_array_v.T, XBX, XBRHS.T), axis = 1)))
    
    return tableau

### Changes on the costs vector c

In [6]:
def sensitivity_costs_vector_c(tableau, original_costs, changed_costs, variables, basic_indexes, non_basic_indexes):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    
    basic_change = False
    non_basic_change = False
    
    for i in range(n_plus_m):
        if original_costs[i] != changed_costs[i]:
            index_change = i
            ck = original_costs[i]
            ck_p = changed_costs[i]
            
            # Non basic change
            if index_change in non_basic_indexes:
                print("\n\033[1mSensitivity analysis: Changes on the costs vector\033[0m")
                print("Detected modification in \033[91mnon basic variable", variables[index_change], "\033[0m")
                non_basic_change = True
                z_minus_c_p = tableau[0, 1 + index_change] + (ck - ck_p)
                tableau[0, 1 + index_change] = z_minus_c_p
                if z_minus_c_p > 0:
                    print("\n[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
                          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]")
                    print("\nThe problem needs the simplex method applied to it beginning with the resultant table")
                    print(tableau)
                    simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))
                else:
                    print("\nThe changed solution remains optimal")
                    print(tableau)
            # Basic change
            else:
                print("\n\033[1mSensitivity analysis: Changes on the costs vector c\033[0m")
                print("Detected modification in \033[94mbasic variable", variables[index_change], "\033[0m")
                print("\n[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
                          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]")
                basic_change = True
                k_relative = 0
                for i in range(len(basic_indexes)):
                    if index_change == basic_indexes[i]:
                        k_relative = i
                        break
                
                row_k_relative = np.ravel(tableau[k_relative + 1, :]) * (ck_p - ck)
                tableau[0, :] += row_k_relative
                tableau[0, index_change + 1] = 0
                
                index_positive = -1
                ZX = np.ravel(tableau[0, 1:n_plus_m + 1])
                for i in range(len(ZX)):
                    if ZX[i] > 0:
                        index_positive = i
                        break
                
                if index_positive in non_basic_indexes:
                    print("\nThe problem needs the simplex method applied to it beginning with the resultant table\n")
                    print(tableau)
                    simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))
                else:
                    print("\nAlthough changes, the problem has the optimal primal table\n")
                    print(tableau)
            break
            

### Changes on the right vector b

In [7]:
def sensitivity_right_vector_b(tableau, A, c, original_b, changed_b, variables, basic_indexes, non_basic_indexes, slack_indexes):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
    
    B = A[:, basic_indexes]
    c_b = c[basic_indexes]
    original_basic_solution = np.linalg.inv(B) @ original_b
    changed_basic_solution = np.linalg.inv(B) @ changed_b
    tableau[1:, -1] = changed_basic_solution
    z = c_b @ np.linalg.inv(B) @ changed_b
    tableau[0, -1] = z
    
    for i in range(len(changed_basic_solution)):
        if original_b[i] != changed_b[i]:
            index_change = i
            print("\n\033[1mSensitivity analysis: Changes on the right vector b\033[0m")
            print("Detected modification in \033[91mentry no.", str(i + 1), "\033[0m")
            print("\n[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
                          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]")
            counter_flag = 0
            for i in range(len(changed_basic_solution)):
                if changed_basic_solution[i] >= 0:
                    counter_flag += 1
            if counter_flag == len(changed_basic_solution):
                print("\nAlthough changes, the problem has the optimal primal table\n")
                print(tableau)
            else:
                print("\nThe problem needs the dual simplex method applied to it beginning with the resultant table\n")
                print(tableau)
                dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))
                

### Changes on the constraints matrix A

In [8]:
def sensitivity_constraint_matrix_A(tableau, A, c, aj, aj_p, variables, basic_indexes, non_basic_indexes):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
    
    index_change = -1
    for i in range(len(basic_indexes + non_basic_indexes)):
        counter_flag = 0
        for j in range(len(basic_indexes)):
            if A[j, i] == aj[j]:
                counter_flag += 1
        if counter_flag == len(basic_indexes):
            index_change = i
            break
    
    if index_change != -1:
        B = A[:, basic_indexes]
        c_b = c[basic_indexes]
        yj_p = np.linalg.inv(B) @ aj_p
        zj_p_minus_cj = c_b @ np.linalg.inv(B) @ aj_p - c[index_change]
        tableau[0, index_change + 1] = zj_p_minus_cj
        tableau[1:, index_change + 1] = yj_p

        if index_change in non_basic_indexes:

            print("\n\033[1mSensitivity analysis: Changes on the constraints matrix A\033[0m")
            print("Detected modification in \033[91mnon basic variable", variables[index_change], "\033[0m")
            print("\n[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
                          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]")
            
            if zj_p_minus_cj <= 0:
                print("\nAlthough changes, the problem has the optimal primal table\n")
                print(tableau)
            else:
                print("\nThe problem needs the dual simplex method applied to it beginning with the resultant table\n")
                print(tableau)
                simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))

        else:

            print("\n\033[1mSensitivity analysis: Changes on the constraints matrix A\033[0m")
            print("Detected modification in \033[94m basic variable", variables[index_change], "\033[0m")
            print("\n[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
                          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]")
            
            k_relative = 0
            for i in range(len(basic_indexes)):
                if index_change == basic_indexes[i]:
                    k_relative = i
                    break

            if yj_p[k_relative] == 0:
                print("\nThe current set of basic vectors no longer forms a basis\n")
                print(tableau)
            else:
                pivot = yj_p[k_relative]          
                tableau[index_change + 1, :] = tableau[index_change + 1, :] / pivot
                for i in range(len(tableau)):
                    if i != k_relative + 1:
                        tableau[i, :] = tableau[i, :] - tableau[i, index_change + 1] * tableau[k_relative + 1, :]
                print("\nAfter pivoting we get the resultant table\n")
                print(tableau)
            

So, once we have already defined the previous function, we are able to apply it.

#### Example 1:
Consider the following problem:

$$min \hspace{0.5cm} -2x_1 + x_2 - x_3$$
$$under \hspace{0.5cm} x_1 + x_2 + x_3 \leq 6$$
$$\hspace{1.4cm} -x_1 + 2x_2 + 0x_3 \leq 4$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

Introducing our slack variables $x_4, x_5$ and then apply the simplex method by tableau format we get the primal optimal table:

|     | $z$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $RHS$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$ | $1$ | $0$ | $-3$ | $-1$ | $-2$ | $0$ | $-12$ |
| $X_{1}$ | $0$ | $1$ | $1$ | $1$ | $1$ | $0$ | $6$ |
| $X_{5}$ | $0$ | $0$ | $3$ | $1$ | $1$ | $1$ | $10$ |

In [9]:
A = np.array([[1, 1, 1, 1, 0],
             [-1, 2, 0, 0, 1]])
b = np.array([6, 4])
c = np.array([-2, 1, -1, 0, 0])

In [10]:
variables = ["x1", "x2", "x3", "x4", "x5"]
basic_indexes = [0, 4]
non_basic_indexes = [1, 2, 3]
slack_indexes = [3, 4]

In [11]:
optimal_tableau = np.array([[1., 0., -3., -1., -2., 0., -12.],
                            [0., 1., 1., 1., 1., 0., 6.],
                            [0., 0., 3., 1., 1., 1., 10.]])

In [12]:
optimal_tableau

array([[  1.,   0.,  -3.,  -1.,  -2.,   0., -12.],
       [  0.,   1.,   1.,   1.,   1.,   0.,   6.],
       [  0.,   0.,   3.,   1.,   1.,   1.,  10.]])

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccc | ccc}
1 & 1 & 2 & 1 & 0 & 0\\
1 & 1 & -1 & 0 & 1 & 0\\
-1 & 1 & 1 & 0 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
9\\
2\\
4\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{B} | c_{N}] =
\left[
\begin{array}{ccc|ccc}
1 & 1 & -4 & 0 & 0 & 0\\
\end{array}
\right]
\end{equation}

##### Costs vector c

\begin{equation} c =
\left[
\begin{array}{ccccc}
-2 & 1 & -1 & 0 & 0\\
\end{array}
\right]
\end{equation}

\begin{equation} c' =
\left[
\begin{array}{ccccc}
-2 & -3 & -1 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [13]:
original_costs = np.array([-2, 1, -1, 0, 0])
changed_costs = np.array([-2, -3, -1, 0, 0])

In [14]:
sensitivity_costs_vector_c(copy.deepcopy(optimal_tableau), copy.deepcopy(original_costs), copy.deepcopy(changed_costs), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))


Sensitivity analysis: Changes on the costs vector
Detected modification in non basic variable x2 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

The problem needs the simplex method applied to it beginning with the resultant table
[[  1.   0.   1.  -1.  -2.   0. -12.]
 [  0.   1.   1.   1.   1.   0.   6.]
 [  0.   0.   3.   1.   1.   1.  10.]]


Iteration  1 
[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]


Entry tableau
[[  1.   0.   1.  -1.  -2.   0. -12.]
 [  0.   1.   1.   1.   1.   0.   6.]
 [  0.   0.   3.   1.   1.   1.  10.]]

k = 1 -> column no. 1 of x_N part ( x2 )
y_k
[1. 3.]

r = 2 -> column no. 2 of x_B part ( x5 )
x_Br
3.3333333333333335

pivot = 3.0

Pivoted tableau
[[  1.       0.       0.      -1.3333  -2.3333  -0.3333 -15.3333]
 [  0.       1.       0.       0.6667   0.6667  -0.3333   2.6667]
 [  0.       0.       1.       0.3333   0.3333   0.3333   3.3333]]

 x2 enters and  x5 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x1, x2, x5, x3, x4 ]


Entry tableau
[[  1.       

\begin{equation} c =
\left[
\begin{array}{ccccc}
-2 & 1 & -1 & 0 & 0\\
\end{array}
\right]
\end{equation}

\begin{equation} c' =
\left[
\begin{array}{ccccc}
0 & 1 & -1 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [15]:
original_costs = np.array([-2, 1, -1, 0, 0])
changed_costs = np.array([0, 1, -1, 0, 0])

In [16]:
sensitivity_costs_vector_c(copy.deepcopy(optimal_tableau), copy.deepcopy(original_costs), copy.deepcopy(changed_costs), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))


Sensitivity analysis: Changes on the costs vector c
Detected modification in basic variable x1 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

The problem needs the simplex method applied to it beginning with the resultant table

[[ 1.  0. -1.  1.  0.  0.  0.]
 [ 0.  1.  1.  1.  1.  0.  6.]
 [ 0.  0.  3.  1.  1.  1. 10.]]


Iteration  1 
[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]


Entry tableau
[[ 1.  0. -1.  1.  0.  0.  0.]
 [ 0.  1.  1.  1.  1.  0.  6.]
 [ 0.  0.  3.  1.  1.  1. 10.]]

k = 2 -> column no. 2 of x_N part ( x3 )
y_k
[1. 1.]

r = 1 -> column no. 1 of x_B part ( x1 )
x_Br
6.0

pivot = 1.0

Pivoted tableau
[[ 1. -1. -2.  0. -1.  0. -6.]
 [ 0.  1.  1.  1.  1.  0.  6.]
 [ 0. -1.  2.  0.  0.  1.  4.]]

 x3 enters and  x1 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x3, x5, x2, x1, x4 ]


Entry tableau
[[ 1. -1. -2.  0. -1.  0. -6.]
 [ 0.  1.  1.  1.  1.  0.  6.]
 [ 0. -1.  2.  0.  0.  1.  4.]]


Optimality reached

The optimal BFS is
[ x3, x5, x2, x1, x4 ] = [ 6.0, 4.0, 0.0,

##### Right vector b

\begin{equation} b = 
\left[
\begin{array}{r}
6\\
4\\
\end{array}
\right]
\end{equation}

\begin{equation} b' = 
\left[
\begin{array}{r}
3\\
4\\
\end{array}
\right]
\end{equation}

In [17]:
original_b = np.array([6, 4])
changed_b = np.array([3, 4])

In [18]:
sensitivity_right_vector_b(copy.deepcopy(optimal_tableau), copy.deepcopy(A), copy.deepcopy(c), copy.deepcopy(original_b), copy.deepcopy(changed_b), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))


Sensitivity analysis: Changes on the right vector b
Detected modification in entry no. 1 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

Although changes, the problem has the optimal primal table

[[ 1.  0. -3. -1. -2.  0. -6.]
 [ 0.  1.  1.  1.  1.  0.  3.]
 [ 0.  0.  3.  1.  1.  1.  7.]]


##### Constraints matrix A

\begin{equation} a_2 = 
\left[
\begin{array}{r}
1\\
2\\
\end{array}
\right]
\end{equation}

\begin{equation} a'_2 = 
\left[
\begin{array}{r}
2\\
5\\
\end{array}
\right]
\end{equation}

In [19]:
aj = np.array([1, 2])
aj_p = np.array([2, 5])

In [20]:
sensitivity_constraint_matrix_A(copy.deepcopy(optimal_tableau), copy.deepcopy(A), copy.deepcopy(c), copy.deepcopy(aj), copy.deepcopy(aj_p), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))


Sensitivity analysis: Changes on the constraints matrix A
Detected modification in non basic variable x2 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

Although changes, the problem has the optimal primal table

[[  1.   0.  -5.  -1.  -2.   0. -12.]
 [  0.   1.   2.   1.   1.   0.   6.]
 [  0.   0.   7.   1.   1.   1.  10.]]


\begin{equation} a_1 = 
\left[
\begin{array}{r}
1\\
-1\\
\end{array}
\right]
\end{equation}

\begin{equation} a'_1 = 
\left[
\begin{array}{r}
0\\
-1\\
\end{array}
\right]
\end{equation}

In [21]:
aj = np.array([1, -1])
aj_p = np.array([0, -1])

In [22]:
sensitivity_constraint_matrix_A(copy.deepcopy(optimal_tableau), copy.deepcopy(A), copy.deepcopy(c), copy.deepcopy(aj), copy.deepcopy(aj_p), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))


Sensitivity analysis: Changes on the constraints matrix A
Detected modification in  basic variable x1 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

The current set of basic vectors no longer forms a basis

[[  1.   2.  -3.  -1.  -2.   0. -12.]
 [  0.   0.   1.   1.   1.   0.   6.]
 [  0.  -1.   3.   1.   1.   1.  10.]]


\begin{equation} a_1 = 
\left[
\begin{array}{r}
1\\
-1\\
\end{array}
\right]
\end{equation}

\begin{equation} a'_1 = 
\left[
\begin{array}{r}
3\\
6\\
\end{array}
\right]
\end{equation}

In [23]:
aj = np.array([1, -1])
aj_p = np.array([3, 6])

In [24]:
sensitivity_constraint_matrix_A(copy.deepcopy(optimal_tableau), copy.deepcopy(A), copy.deepcopy(c), copy.deepcopy(aj), copy.deepcopy(aj_p), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))


Sensitivity analysis: Changes on the constraints matrix A
Detected modification in  basic variable x1 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

After pivoting we get the resultant table

[[ 1.      0.     -1.6667  0.3333 -0.6667  0.     -4.    ]
 [ 0.      1.      0.3333  0.3333  0.3333  0.      2.    ]
 [ 0.      0.      0.     -2.     -2.      1.     -8.    ]]


#### Exercise 6.49

Consider the following problem:

$$max \hspace{0.5cm} 2x_1 + x_2 - x_3$$
$$under \hspace{0.5cm} x_1 + 2x_2 + x_3 \leq 8$$
$$\hspace{1.4cm} -x_1 + x_2 - 2x_3 \leq 4$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

Introducing our slack variables $x_4, x_5$ and then apply the simplex method by tableau format we get the primal optimal table:

|     | $z$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $RHS$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$ | $1$ | $0$ | $3$ | $3$ | $2$ | $0$ | $16$ |
| $X_{1}$ | $0$ | $1$ | $2$ | $1$ | $1$ | $0$ | $8$ |
| $X_{5}$ | $0$ | $0$ | $3$ | $-1$ | $1$ | $1$ | $12$ |

For mere convenience, we rewrite the problem as

$$min \hspace{0.5cm} -2x_1 - x_2 + x_3 + 0x_4 + 0x_5$$
$$under \hspace{0.5cm} x_1 + 2x_2 + x_3 + x_4 + 0 x_5 = 8$$
$$\hspace{1.4cm} - x_1 + x_2 - 2x_3 + 0x_4 + x_5 = 4$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

In [25]:
A = np.array([[1, 2, 1, 1, 0],
             [-1, 1, -2, 0, 1]])
b = np.array([8, 4])
c = np.array([-2, -1, 1, 0, 0])

In [26]:
variables = ["x1", "x2", "x3", "x4", "x5"]
basic_indexes = [0, 4]
non_basic_indexes = [1, 2, 3]
slack_indexes = [3, 4]

In [27]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [28]:
tableau

array([[  1.,   0.,  -3.,  -3.,  -2.,   0., -16.],
       [  0.,   1.,   2.,   1.,   1.,   0.,   8.],
       [  0.,   0.,   3.,  -1.,   1.,   1.,  12.]])

So introducing our slack variables $x_4, x_5$ and then apply the simplex method by tableau format we get the primal optimal table:

|     | $z$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $RHS$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$ | $1$ | $0$ | $-3$ | $-3$ | $-2$ | $0$ | $-16$ |
| $X_{1}$ | $0$ | $1$ | $2$ | $1$ | $1$ | $0$ | $8$ |
| $X_{5}$ | $0$ | $0$ | $3$ | $-1$ | $1$ | $1$ | $12$ |

In [29]:
optimal_tableau = np.array([[1., 0., -3., -3., -2., 0., -16.],
                            [0., 1., 2., 1., 1., 0., 8.],
                            [0., 0., 3., -1., 1., 1., 12.]])

In [30]:
optimal_tableau

array([[  1.,   0.,  -3.,  -3.,  -2.,   0., -16.],
       [  0.,   1.,   2.,   1.,   1.,   0.,   8.],
       [  0.,   0.,   3.,  -1.,   1.,   1.,  12.]])

##### b) Costs vector c

\begin{equation} c =
\left[
\begin{array}{ccccc}
-2 & -1 & 1 & 0 & 0\\
\end{array}
\right]
\end{equation}

\begin{equation} c' =
\left[
\begin{array}{ccccc}
-2 & -5 & 1 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [31]:
original_costs = np.array([-2, -1, 1, 0, 0])
changed_costs = np.array([-2, -5, 1, 0, 0])

In [32]:
sensitivity_costs_vector_c(copy.deepcopy(optimal_tableau), copy.deepcopy(original_costs), copy.deepcopy(changed_costs), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))


Sensitivity analysis: Changes on the costs vector
Detected modification in non basic variable x2 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

The problem needs the simplex method applied to it beginning with the resultant table
[[  1.   0.   1.  -3.  -2.   0. -16.]
 [  0.   1.   2.   1.   1.   0.   8.]
 [  0.   0.   3.  -1.   1.   1.  12.]]


Iteration  1 
[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]


Entry tableau
[[  1.   0.   1.  -3.  -2.   0. -16.]
 [  0.   1.   2.   1.   1.   0.   8.]
 [  0.   0.   3.  -1.   1.   1.  12.]]

k = 1 -> column no. 1 of x_N part ( x2 )
y_k
[2. 3.]

r = 1 -> column no. 1 of x_B part ( x1 )
x_Br
4.0

pivot = 2.0

Pivoted tableau
[[  1.   -0.5   0.   -3.5  -2.5   0.  -20. ]
 [  0.    0.5   1.    0.5   0.5   0.    4. ]
 [  0.   -1.5   0.   -2.5  -0.5   1.    0. ]]

 x2 enters and  x1 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x2, x5, x1, x3, x4 ]


Entry tableau
[[  1.   -0.5   0.   -3.5  -2.5   0.  -20. ]
 [  0.    0.5   1.    0.5   0.5   0.    4. ]
 

##### c) Constraints matrix A

\begin{equation} a_3 = 
\left[
\begin{array}{r}
1\\
-2\\
\end{array}
\right]
\end{equation}

\begin{equation} a'_3 = 
\left[
\begin{array}{r}
1\\
1\\
\end{array}
\right]
\end{equation}

In [33]:
aj = np.array([1, -2])
aj_p = np.array([1, 1])

In [34]:
sensitivity_constraint_matrix_A(copy.deepcopy(optimal_tableau), copy.deepcopy(A), copy.deepcopy(c), copy.deepcopy(aj), copy.deepcopy(aj_p), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))


Sensitivity analysis: Changes on the constraints matrix A
Detected modification in non basic variable x3 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

Although changes, the problem has the optimal primal table

[[  1.   0.  -3.  -3.  -2.   0. -16.]
 [  0.   1.   2.   1.   1.   0.   8.]
 [  0.   0.   3.   2.   1.   1.  12.]]
